In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os  
import glob
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow import keras
from pathlib import Path
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from keras.layers import Lambda


In [45]:
latent_dim = 64 

class Encoder(Layer):
  def __init__(self):
    super(Encoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
    
      layers.Input(shape=(24, 24, 1)), 
      layers.Conv2D(16, (6,6), activation='relu', padding='valid', strides=1),
      layers.Conv2D(8,(2,2), activation='relu', padding='valid', strides=1)])

  def call(self, x):
    encoded = self.encoder(x)
    return encoded



In [46]:
class Decoder(Layer):
  def __init__(self):
    super(Decoder, self).__init__()
    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=(2,2), strides=1, activation='relu',padding='valid'),
      layers.Conv2DTranspose(16, kernel_size=(6,6), strides=1, activation='relu',padding='valid'),
      layers.Conv2D(1, kernel_size=(1,1), activation='sigmoid', padding='valid')])

  def call(self, x):
    decoded = self.decoder(x)
    return decoded


In [47]:
class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder,self).__init__()
        self.Autoencoder=tf.keras.Sequential([
            Encoder(),
            Decoder()
        ])
    def call(self,x):
        return self.Autoencoder(x)

In [48]:
autoencoder = Autoencoder()

In [49]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())


In [50]:
Full=tf.io.gfile.listdir('24TaxonFull')

In [51]:
Missing=tf.io.gfile.listdir('24TaxonData')

In [52]:
FullDataset=[]
for i in Full:
    name, extension = os.path.splitext('24TaxonFull/'+str(i))
    if extension=='.csv':
        ReadPanda=pd.read_csv('24TaxonFull/'+str(i))
        FullDataset.append(ReadPanda)



In [53]:
MissingDataset=[]
for i in Missing:

    name, extension = os.path.splitext('24TaxonData/'+str(i))
    if extension=='.csv':
        ReadPanda=pd.read_csv('24TaxonData/'+str(i))
        MissingDataset.append(ReadPanda)



In [54]:
np.shape(np.array(FullDataset))
FullDataset=np.array(FullDataset)
np.save('FullDataset',FullDataset)
np.shape(np.array(MissingDataset))
MissingDataset=np.array(MissingDataset)
np.save('MissingDataset',MissingDataset)

In [55]:
np.shape(MissingDataset)

(1000, 24, 24)

In [56]:
FullDataSet=FullDataset[...,tf.newaxis]
np.shape(FullDataSet)

(1000, 24, 24, 1)

In [57]:
MissingDataSet=MissingDataset[...,tf.newaxis]
np.shape(MissingDataSet)

(1000, 24, 24, 1)

In [58]:
if os.path.isdir("DistanceEncoderWeights"):
    pass
else:
    autoencoder.fit(MissingDataSet,FullDataSet,
                    epochs=1000,
                    shuffle=True,
                   batch_size=5)
    autoencoder.save("DistanceEncoderWeights")

Epoch 1/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0439
Epoch 2/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0038
Epoch 3/100
200/200 [==============================] - 2s 8ms/step - loss: 0.0015
Epoch 4/100
200/200 [==============================] - 2s 8ms/step - loss: 9.5838e-04
Epoch 5/100
200/200 [==============================] - 2s 8ms/step - loss: 7.4735e-04
Epoch 6/100
200/200 [==============================] - 2s 8ms/step - loss: 5.8813e-04
Epoch 7/100
200/200 [==============================] - 2s 8ms/step - loss: 4.5880e-04
Epoch 8/100
200/200 [==============================] - 2s 8ms/step - loss: 3.5958e-04
Epoch 9/100
200/200 [==============================] - 2s 9ms/step - loss: 2.7188e-04
Epoch 10/100
200/200 [==============================] - 2s 8ms/step - loss: 2.2214e-04
Epoch 11/100
200/200 [==============================] - 2s 8ms/step - loss: 1.9680e-04
Epoch 12/100
200/200 [==============================] - 2s 8ms/s

In [66]:
autoencoder.evaluate(MissingDataSet,FullDataSet,20)

50/50 [==============================] - 0s 9ms/step - loss: 2.1220e-05


2.1219560949248262e-05

In [67]:
Test=MissingDataset[tf.newaxis,0,...,tf.newaxis]
np.shape(Test)

(1, 24, 24, 1)

In [68]:

Export=autoencoder.predict(Test)

In [62]:
Export=pd.DataFrame(np.reshape(Export,(24,24)))

In [63]:
Export=Export-np.diag(np.diag(Export))

In [64]:
Export.to_csv("Example_Output.csv",index=False)

In [65]:
Export

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.000000,0.003623,0.142437,0.145649,0.142419,0.149034,0.136620,0.139350,0.139793,0.140135,...,0.139119,0.139266,0.139463,0.139042,0.144736,0.150596,0.146167,0.132792,0.137168,0.159829
1,0.003123,0.000000,0.141730,0.144568,0.140405,0.149824,0.136857,0.126401,0.135979,0.138560,...,0.142382,0.142423,0.140747,0.138226,0.145138,0.150843,0.146769,0.133890,0.138145,0.160025
2,0.143705,0.144996,0.000000,0.009571,0.101024,0.109476,0.046051,0.039875,0.063783,0.072136,...,0.090224,0.088554,0.086720,0.081333,0.094052,0.093878,0.095601,0.147290,0.138981,0.153711
3,0.143658,0.143738,0.009627,0.000000,0.100538,0.109465,0.044087,0.040454,0.056715,0.061589,...,0.089167,0.086933,0.088307,0.086962,0.094011,0.089735,0.093568,0.145506,0.139421,0.153051
4,0.144417,0.142600,0.099644,0.099534,0.000000,0.046990,0.094092,0.082044,0.091529,0.092154,...,0.083691,0.085006,0.086395,0.085135,0.093065,0.091127,0.093517,0.146183,0.137286,0.151447
5,0.149461,0.148548,0.103876,0.105187,0.044729,0.000000,0.098451,0.090514,0.092196,0.095699,...,0.088627,0.088910,0.088574,0.088244,0.095946,0.093876,0.099595,0.148696,0.138001,0.153592
6,0.132525,0.136172,0.047774,0.047564,0.094628,0.081407,0.000000,0.025316,0.057896,0.061415,...,0.084096,0.083811,0.083311,0.083743,0.087158,0.088506,0.091926,0.145874,0.136436,0.152200
7,0.128526,0.126289,0.041428,0.041031,0.076114,0.084758,0.025361,0.000000,0.041557,0.057206,...,0.083852,0.084457,0.083336,0.084003,0.088933,0.089882,0.090137,0.146843,0.136917,0.153001
8,0.139786,0.136889,0.066687,0.059244,0.090650,0.092454,0.060210,0.049687,0.000000,0.012893,...,0.084513,0.085273,0.084371,0.084017,0.090274,0.090134,0.090700,0.146618,0.137030,0.153174
9,0.140636,0.139927,0.072664,0.063231,0.087718,0.090364,0.061650,0.060400,0.013361,0.000000,...,0.082055,0.084556,0.083699,0.083227,0.089479,0.089262,0.090339,0.146936,0.137269,0.153111
